In [30]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

#Get datasets
ccgs_92377=pd.read_csv('ccgs_92377_clean.csv',low_memory=False)
diff=pd.read_csv('diff_2014_2017.csv')
diff.rename(columns = {'Unnamed: 0':'CCG'}, inplace = True)

ccgs_list=ccgs_92377['CCG'].unique().tolist() #list of ccg names
ccgs_ts=pd.DataFrame() #dataframe for ccgs time series
for index, value in enumerate (ccgs_list):
        indicator_ccg=ccgs_92377[ccgs_92377['CCG']==value]
        indicator_ccg_values=indicator_ccg.loc[:,'Value']
        ccgs_ts [value]=pd.Series()
        ccgs_ts [value]=indicator_ccg_values.values #store the indicator data for all ccgs
        
#datetime index for the ccgs
datetime_index=pd.to_datetime(ccgs_92377['Timeperiod'].unique(), format='%Y-%m-%d')
ccgs_ts=ccgs_ts.set_index(datetime_index)

In [31]:
ccg_to_region=ccgs_92377.loc[ccgs_92377['Timeperiod']=='2016-04-01']
ccg_to_region=ccg_to_region[['CCG','Region','AreaCode']]
ccg_to_diff=diff[['CCG','%diff.']]

region_dict = dict(zip(ccg_to_region['CCG'], ccg_to_region['Region']))
area_dict = dict(zip(ccg_to_region['CCG'], ccg_to_region['AreaCode']))
diff_dict=dict(zip (ccg_to_diff['CCG'],ccg_to_diff['%diff.']))

In [34]:
#melt ccgs
ccgs_ts['Time_period']=ccgs_ts.index
ccgs_ts_m=pd.melt(frame=ccgs_ts, id_vars='Time_period')
ccgs_ts_m.rename(columns = {'variable':'CCG'}, inplace = True)

In [35]:
#build Regions column
regions_list=[] #list for regions 
areas_list=[] #list for area codes
diff_list=[] #list for diff (reduction) values
target_class_list=[]
for label, row in ccgs_ts_m.iterrows() :
        regions_list=regions_list+[region_dict[row['CCG']]]
        areas_list=areas_list+[area_dict[row['CCG']]]
        diff_list=diff_list + [diff_dict[row['CCG']]]
ccgs_ts_m['Region']=regions_list
ccgs_ts_m['AreaCode']=areas_list
ccgs_ts_m['%diff.']=diff_list

ccgs_ts_m=ccgs_ts_m[['Time_period', 'CCG', 'Region','AreaCode','value','%diff.']]

In [39]:
#write ccvs
ccgs_ts_m.to_csv('ccgs_time_series.csv')